In [1]:
from neuralnetlib.models import Transformer
from neuralnetlib.preprocessing import Tokenizer, pad_sequences
from neuralnetlib.optimizers import Adam

In [2]:
fr_sentences = [
    "bonjour.",
    "au revoir.",
    "merci beaucoup.",
    "s'il vous plaît.",
    "comment allez-vous ?",
    "je vais bien.",
    "je suis fatigué.",
    "je suis content.",
    "quel est votre nom ?",
    "mon nom est Jean.",
    "enchanté de vous rencontrer.",
    "bonne journée.",
    "bonne soirée.",
    "à demain.",
    "j'aime le café.",
    "je n'aime pas le thé.",
    "quelle heure est-il ?",
    "il est trois heures.",
    "où est la gare ?",
    "la gare est près d'ici.",
    "combien ça coûte ?",
    "c'est trop cher.",
    "parlez-vous anglais ?",
    "un peu.",
    "je ne comprends pas.",
    "pouvez-vous répéter ?",
    "je suis désolé.",
    "pas de problème.",
    "bon appétit.",
    "à votre santé.",
    "j'ai faim.",
    "j'ai soif.",
    "il fait beau aujourd'hui.",
    "il pleut.",
    "il fait froid.",
    "il fait chaud.",
    "je travaille ici.",
    "où habitez-vous ?",
    "j'habite à Paris.",
    "quel âge avez-vous ?",
    "j'ai vingt-cinq ans.",
    "avez-vous des frères et sœurs ?",
    "j'ai une sœur.",
    "j'ai un chat.",
    "j'aime voyager.",
    "je suis étudiant.",
    "je suis professeur.",
    "au secours !",
    "joyeux anniversaire !",
    "félicitations !"
]

en_sentences = [
    "hello.",
    "goodbye.",
    "thank you very much.",
    "please.",
    "how are you?",
    "i am fine.",
    "i am tired.",
    "i am happy.",
    "what is your name?",
    "my name is John.",
    "nice to meet you.",
    "have a nice day.",
    "have a good evening.",
    "see you tomorrow.",
    "i like coffee.",
    "i don't like tea.",
    "what time is it?",
    "it is three o'clock.",
    "where is the train station?",
    "the station is nearby.",
    "how much is it?",
    "it's too expensive.",
    "do you speak english?",
    "a little.",
    "i don't understand.",
    "can you repeat?",
    "i am sorry.",
    "no problem.",
    "enjoy your meal.",
    "cheers.",
    "i am hungry.",
    "i am thirsty.",
    "the weather is nice today.",
    "it's raining.",
    "it's cold.",
    "it's hot.",
    "i work here.",
    "where do you live?",
    "i live in Paris.",
    "how old are you?",
    "i am twenty-five years old.",
    "do you have brothers and sisters?",
    "i have a sister.",
    "i have a cat.",
    "i like to travel.",
    "i am a student.",
    "i am a teacher.",
    "help!",
    "happy birthday!",
    "congratulations!"
]

In [3]:
fr_tokenizer = Tokenizer()
en_tokenizer = Tokenizer()

fr_tokenizer.fit_on_texts(fr_sentences, preprocess_ponctuation=True)
en_tokenizer.fit_on_texts(en_sentences, preprocess_ponctuation=True)

In [4]:
x_train = fr_tokenizer.texts_to_sequences(fr_sentences, preprocess_ponctuation=True)
y_train = en_tokenizer.texts_to_sequences(en_sentences, preprocess_ponctuation=True)

max_len_x = max(len(seq) for seq in x_train)
max_len_y = max(len(seq) for seq in y_train)
max_seq_len = max(max_len_x, max_len_y)

vocab_size_fr = len(fr_tokenizer.word_index)
vocab_size_en = len(en_tokenizer.word_index)
max_vocab_size = max(vocab_size_fr, vocab_size_en)

In [5]:
# Verify all data
print(f"vocab_size_en: {vocab_size_en}, vocab_size_fr: {vocab_size_fr}")
print(f"max_len_x: {max_len_x}, max_len_y: {max_len_y}, max_vocab_size: {max_vocab_size}, max_seq_len: {max_seq_len}")
print("French sentences:")
print(fr_sentences)
print("English sentences:")
print(en_sentences)
print("French tokenizer:")
print(fr_tokenizer.word_index)
print("English tokenizer:")
print(en_tokenizer.word_index)

vocab_size_en: 91, vocab_size_fr: 103
max_len_x: 8, max_len_y: 8, max_vocab_size: 103, max_seq_len: 8
French sentences:
['bonjour.', 'au revoir.', 'merci beaucoup.', "s'il vous plaît.", 'comment allez-vous ?', 'je vais bien.', 'je suis fatigué.', 'je suis content.', 'quel est votre nom ?', 'mon nom est Jean.', 'enchanté de vous rencontrer.', 'bonne journée.', 'bonne soirée.', 'à demain.', "j'aime le café.", "je n'aime pas le thé.", 'quelle heure est-il ?', 'il est trois heures.', 'où est la gare ?', "la gare est près d'ici.", 'combien ça coûte ?', "c'est trop cher.", 'parlez-vous anglais ?', 'un peu.', 'je ne comprends pas.', 'pouvez-vous répéter ?', 'je suis désolé.', 'pas de problème.', 'bon appétit.', 'à votre santé.', "j'ai faim.", "j'ai soif.", "il fait beau aujourd'hui.", 'il pleut.', 'il fait froid.', 'il fait chaud.', 'je travaille ici.', 'où habitez-vous ?', "j'habite à Paris.", 'quel âge avez-vous ?', "j'ai vingt-cinq ans.", 'avez-vous des frères et sœurs ?', "j'ai une sœur."

In [ ]:
model = Transformer(
    vocab_size=max_vocab_size,
    d_model=64,
    n_heads=4,
    n_encoder_layers=1,
    n_decoder_layers=1,
    d_ff=128,
    dropout_rate=0.1,
    random_state=42
)


model.compile(
    loss_function='sequencecrossentropy',
    optimizer=Adam(learning_rate=0.0005),
    verbose=True
)

Transformer(
  vocab_size=107,
  d_model=64,
  n_heads=4,
  n_encoder_layers=1,
  n_decoder_layers=1,
  d_ff=128,
  dropout_rate=0.1,
  max_sequence_length=512
)


In [7]:
x_train_padded, y_train_padded = model.prepare_data(x_train, y_train)

In [8]:
history = model.fit(
    x_train_padded, y_train_padded,
    epochs=50,
    batch_size=16,
    verbose=True
)

[==============================] 100% Epoch 1/50 - loss: 11.5804 -  - 7.56s
[==============================] 100% Epoch 2/50 - loss: 10.8054 -  - 7.40s
[==============================] 100% Epoch 3/50 - loss: 8.1660 -  - 7.42s
[==============================] 100% Epoch 4/50 - loss: 5.0352 -  - 7.37s
[==============================] 100% Epoch 5/50 - loss: 1.4511 -  - 7.36s
[==============================] 100% Epoch 6/50 - loss: 0.3899 -  - 7.35s
[==============================] 100% Epoch 7/50 - loss: 0.1863 -  - 7.35s
[==============================] 100% Epoch 8/50 - loss: 0.1264 -  - 7.38s
[==============================] 100% Epoch 9/50 - loss: 0.0901 -  - 7.35s
[==============================] 100% Epoch 10/50 - loss: 0.0717 -  - 7.36s
[==============================] 100% Epoch 11/50 - loss: 0.0599 -  - 7.35s
[==============================] 100% Epoch 12/50 - loss: 0.0518 -  - 7.38s
[==============================] 100% Epoch 13/50 - loss: 0.0462 -  - 7.30s
[==================

In [ ]:
def translate(sentence: str, model, fr_tokenizer, en_tokenizer) -> str:
    print("\nDebug tokenization:")
    print(f"Original sentence: {sentence}")
    tokens = fr_tokenizer.texts_to_sequences([sentence], preprocess_ponctuation=True)[0]
    print(f"Tokens after initial tokenization: {tokens}")
    print(f"Corresponding words: {fr_tokenizer.sequences_to_texts([tokens])[0]}")
    
    tokens = fr_tokenizer.encode_special_tokens([tokens])[0]
    print(f"Tokens after special tokens: {tokens}")
    
    padded = pad_sequences([tokens], max_length=max_len_x, padding='post', 
                          pad_value=fr_tokenizer.word_index[fr_tokenizer.pad_token])
    print(f"Padded sequence: {padded}")
    
    pred = model.predict(padded, max_length=max_seq_len, beam_size=10, temperature=1.0)[0]
    print(f"Raw prediction: {pred}")
    
    words = []
    print("\nToken conversion:")
    for idx in pred[1:]:  # Skip SOS
        if idx == en_tokenizer.word_index[en_tokenizer.eos_token]:
            print(f"Found EOS token at index {idx}")
            break
        if idx in [en_tokenizer.word_index[t] for t in [en_tokenizer.pad_token, 
                                                       en_tokenizer.unk_token, 
                                                       en_tokenizer.sos_token]]:
            print(f"Skipping special token {idx}")
            continue
        word = en_tokenizer.index_word.get(idx, en_tokenizer.unk_token)
        print(f"Index {idx} maps to word: {word}")
        words.append(word)
    
    result = " ".join(words)
    print(f"\nFinal translation: {result}")
    return result

# Test avec des phrases du jeu d'entraînement
test_sentences = [
    "je vais bien.",        # Phrase simple
    "comment allez-vous ?", # Phrase interrogative
    "bonjour."             # Phrase très courte
]

print("Vocabulary sizes:")
print(f"French vocab size: {len(fr_tokenizer.word_index)}")
print(f"English vocab size: {len(en_tokenizer.word_index)}")

for sent in test_sentences:
    print("\n" + "="*50)
    print(f"Testing: {sent}")
    translation = translate(sent, model, fr_tokenizer, en_tokenizer)

Vocabulary sizes:
French vocab size: 103
English vocab size: 91

Testing: je vais bien.

Debug tokenization:
Original sentence: je vais bien.
Tokens after initial tokenization: [1, 33, 34, 101]
Corresponding words: je vais bien <UNK>
Tokens after special tokens: [102, 1, 33, 34, 101, 103]
Padded sequence: [[102   1  33  34 101 103   0   0]]
Raw prediction: [105  19]

Token conversion:
Index 19 maps to word: name

Final translation: name

Testing: comment allez-vous ?

Debug tokenization:
Original sentence: comment allez-vous ?
Tokens after initial tokenization: [31, 32, 101, 2, 101]
Corresponding words: comment allez <UNK> vous <UNK>
Tokens after special tokens: [102, 31, 32, 101, 2, 101, 103]
Padded sequence: [[102  31  32 101   2 101 103   0]]
Raw prediction: [105  19]

Token conversion:
Index 19 maps to word: name

Final translation: name

Testing: bonjour.

Debug tokenization:
Original sentence: bonjour.
Tokens after initial tokenization: [25, 101]
Corresponding words: bonjour <UNK